Nowe

In [3]:
import json, os, random, time
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim

In [68]:
def create_dataframe_points(xy_from=0, xy_to = 50, num_points = 175):
    """ Function for generation sample data on 2D space.

        Returns:
            list: two lists of tuples.
            First constains only x and y, second contains x, y and weight of parcel.
    """
    data = np.random.uniform(xy_from, xy_to, size=(num_points,2))
    depot = np.array([xy_to/2, xy_to/2])

    weights = [1]*int(0.5429*num_points) + [2]*int(0.2571*num_points) + [3]*int(0.1429*num_points) + [4]*int(0.0571*num_points)
    if len(weights) < num_points:
        weights.extend([1]*(num_points - len(weights)))
    np.random.shuffle(weights)
    weights = np.array([weights])

    X = np.concatenate((data, weights.T), axis=1)
    df = pd.DataFrame({
            "x": X[:][:,0],
            "y": X[:][:,1],
            "weight": X[:][:,2]     
            })
    df.iloc[0] = [depot[0], depot[1], 1.0]

    return [[(float(df.loc[index, 'x']), float(df.loc[index, 'y'])) for index, rowb in df.iterrows()],
            [(float(df.loc[index, 'x']), float(df.loc[index, 'y']), float(df.loc[index, 'weight'])) for index, rowb in df.iterrows()]]

In [69]:
create_dataframe_points()

[[(25.0, 25.0),
  (24.43150602472215, 43.30454923000648),
  (12.760842973523594, 27.74843572569285),
  (37.78459659227253, 42.29386551392722),
  (9.506579668414794, 36.77113141511682),
  (10.698899193465355, 2.8529157626580206),
  (3.4133658402003886, 16.052662532263074),
  (5.234753979257512, 33.6787104474354),
  (42.31826644581182, 30.80489132250861),
  (46.755516321857726, 33.2658952409682),
  (34.18109653998153, 5.172581933388032),
  (18.76726816729084, 38.62475288785643),
  (32.70387439133475, 2.196583136688596),
  (11.662277685202065, 42.96967354353219),
  (24.173485360743186, 30.83874009205687),
  (37.15268560636817, 9.214162205676296),
  (22.019913250132127, 6.238465450374953),
  (27.853608102333144, 44.026228948135696),
  (19.622000803321683, 1.1863246986856368),
  (44.84094537876629, 19.141318829937887),
  (37.70903157455091, 35.00979795776825),
  (18.579767762846842, 17.270696171668583),
  (22.2511342044084, 37.141563010394584),
  (24.395509036359286, 15.001330180766582),
  

In [6]:
os.chdir('..')
os.chdir('Data')

'c:\\Users\\user\\Desktop\\MDP\\courier_logistics\\Data'

In [62]:
os.getcwd()

'c:\\Users\\user\\Desktop\\MDP\\courier_logistics\\Data'

# Version that choices randomly from json

In [117]:
def create_random_addresses(num_points = 175):
    """ Generates data for the problem. 
    
    Args:
        num_points (int): Number of orders
    
    Returns:
        list: Exact addresses of orders
    """
    os.chdir('..')
    os.chdir('Data')
    f = open('data.json', encoding="utf8")
    # returns JSON object as a dictionary
    data = json.load(f)

    street = random.choices(list(data.items()), k=num_points)
    return [i[0] + ' ' + random.choice(i[1]) for i in street]

In [118]:
def geopy_addresses(address_list):
    """Function transforms addresses to be compatible with geopy input.

    Args:
        address_list (list): Exact addresses of orders
    
    Returns:
        list: Exact addresses of orders
    """
    address_list = [x.strip("ul. ") for x in address_list]
    address_list = [x + ", Wrocław, Poland" for x in address_list]
    
    return address_list

In [168]:
test = create_random_addresses(2)
test.append('ul. Hanny Wałkówskiej 4')
test

['ul. Bukowa 8a', 'ul. Grażyny Bacewicz 45', 'ul. Hanny Wałkówskiej 4']

In [169]:
test = geopy_addresses(test)

In [170]:
test

['Bukowa 8a, Wrocław, Poland',
 'Grażyny Bacewicz 45, Wrocław, Poland',
 'Hanny Wałkówskiej 4, Wrocław, Poland']

In [213]:
def filter_addresses(locations):
    """Function filters list with addresses given by user or generated via create_order_coords function.

    Args:
        locations (list): list with addresses
    
    Returns:
        list: addresses that not returns None when using geopy
                for eg. ['Bukowa 8a, Wrocław, Poland', 'Grażyny Bacewicz 45, Wrocław, Poland']
    """
    not_none_address = []
    app = Nominatim(user_agent="t")

    for i, address in enumerate(locations):
        tmp = app.geocode(address)
        if tmp is not None:
            not_none_address.append(address)

        if i % 1000 == 0:
            print(i)

    return not_none_address

In [172]:
app = Nominatim(user_agent="tmp")
print(app.geocode('Rycerska 17, Wrocław, Poland'))

17, Rycerska, Psie Pole, Osiedle Psie Pole-Zawidawie, Wrocław, województwo dolnośląskie, 51-313, Polska


In [173]:
print(app.geocode('Hanny Wałkówskiej 4, Wrocław, Poland'))

None


In [174]:
test = filter_addresses(test)

In [207]:
test

['Bukowa 8a, Wrocław, Poland', 'Grażyny Bacewicz 45, Wrocław, Poland']

In [206]:
[app.geocode(address) for address in test]

[Location(8a, Bukowa, Nowe Żerniki, Osiedle Leśnica, Wrocław, województwo dolnośląskie, 54-115, Polska, (51.1380314, 16.9241377, 0.0)), Location(45, Grażyny Bacewicz, Osiedle „Na Jagodnie”, Jagodno, Osiedle Jagodno, Wrocław, województwo dolnośląskie, 52-131, Polska, (51.0602864, 17.0487844, 0.0))]


In [176]:
def read_coords_geopy(address_list):
    """Find latitude and longitude of order.

    Args:
        address_list (list): Exact addresses of orders

    Returns:
        list: list of tuples that contains coordinates of order
    """
    app = Nominatim(user_agent="tmp")

    df = pd.DataFrame({"lat": [],
                       "lon": []})
    
    locations = [app.geocode(address) for address in address_list]

    for i in range(len(locations)):
        df.loc[len(df.index)] = [locations[i][1][0], locations[i][1][1]]

    return [(float(df.loc[index, 'lat']), float(df.loc[index, 'lon']))
            for index, rowb in df.iterrows()]

In [208]:
read_coords_geopy(test)

[(51.1380314, 16.9241377), (51.0602864, 17.0487844)]

# Zmiana słownika z ulicą i numerami na listę adresów

In [185]:
# trzeba być w folderze \\courier_logistics\\Data
os.getcwd()

'c:\\Users\\user\\Desktop\\MDP\\courier_logistics\\Data'

In [65]:
f = open('data.json', encoding="utf8")
# returns JSON object as a dictionary
data = json.load(f)

In [178]:
all_scrapped_data = []
for i in range(len(data)):
    for j in range(len(list(data.items())[i][1])):
        all_scrapped_data.append([list(data.items())[i][0], list(data.items())[i][1][j]])

In [179]:
all_scrapped_data = [' '.join(element) for element in all_scrapped_data]

In [180]:
all_scrapped_data[0]

'ul. 3 Maja 1b'

In [181]:
all_scrapped_data = [x.strip("ul. ") for x in all_scrapped_data]
all_scrapped_data = [x + ", Wrocław, Poland" for x in all_scrapped_data]

In [184]:
len(all_scrapped_data)

71506

In [189]:
%pip install pyyaml


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [192]:
import yaml

In [200]:
def save_working_addresses(address_list):
    """Save good working addresses to yaml file

    Args:
        address_list (list): list with addresses
    
    Returns:
        .yaml file
    """

    data_folder = os.getcwd()
    address = filter_addresses(address_list)
    
    with open(os.path.join(data_folder, "data.yaml"), 'w', encoding='utf-8') as f:
        yaml.dump(address, f, default_flow_style=False, allow_unicode=True)


In [201]:
test

['Bukowa 8a, Wrocław, Poland', 'Grażyny Bacewicz 45, Wrocław, Poland']

In [202]:
save_working_addresses(test)

Bukowa 8a, Wrocław, Poland
Grażyny Bacewicz 45, Wrocław, Poland


# Otwieranie yamla i znajdowanie listy z tuplami

In [210]:
with open("tmp.yaml", 'r', encoding='utf-8') as f:
    data = yaml.safe_load(f, )

In [211]:
data

['Bukowa 8a, Wrocław, Poland', 'Grażyny Bacewicz 45, Wrocław, Poland']

In [212]:
read_coords_geopy(data)

[(51.1380314, 16.9241377), (51.0602864, 17.0487844)]

In [ ]:
xy_from = 0
xy_to = 50
num_points = 45
weight = np.array([1, 2, 3, 4])
prob = [0.5, 0.35, 0.1, 0.05]
depot = np.array([25, 25])
r_choice = np.array([np.random.choice(weight, size=num_points, p=prob)])

In [ ]:
len(r_choice[0])

45

In [ ]:
data = np.random.uniform(xy_from, xy_to, size=(num_points,2)) # Przykładowe punkty w 2D
distance_to_depo = np.array([([np.math.dist(data[i], depot) for i in range(num_points)])])
data

array([[16.36493497,  7.99023574],
       [18.08080239, 24.2991859 ],
       [ 4.69736449, 46.91279652],
       [14.99269607, 21.48593429],
       [10.92344321, 15.93515432],
       [11.07578807,  3.60677961],
       [ 1.19932186, 29.52971747],
       [30.90226475, 46.06673004],
       [15.82628081, 40.35989066],
       [26.06036465, 42.79390812],
       [13.38768829, 29.37890241],
       [ 8.89571294,  9.03453848],
       [40.42682253, 44.50532569],
       [ 1.38683311, 32.57342192],
       [24.16387837, 32.16909815],
       [43.35149287, 42.61992229],
       [16.13443794, 46.22067482],
       [ 4.05765341, 42.56382895],
       [41.82165156, 19.8126587 ],
       [24.39859357,  8.29449183],
       [ 1.97061404, 13.9533137 ],
       [37.50405191, 33.66919146],
       [14.71824048, 33.66397087],
       [12.23474713, 27.20260482],
       [ 1.05516507,  0.98572569],
       [42.98148702,  4.15348274],
       [10.58040829, 22.18599546],
       [ 2.12348346, 23.06307427],
       [32.99250985,

In [ ]:
X = np.concatenate((r_choice.T, data), axis=1)
X

array([[ 1.        , 15.0906866 , 43.36069784],
       [ 2.        , 38.6808564 , 15.35154212],
       [ 2.        ,  5.64436072, 32.98566816],
       [ 2.        ,  2.93605973,  9.05343599],
       [ 1.        ,  5.14142201, 49.56532658],
       [ 2.        , 25.24210465, 49.38968358],
       [ 2.        , 16.42976652, 47.93440452],
       [ 2.        , 24.89132082, 48.13590617],
       [ 2.        , 11.94144141, 43.60858955],
       [ 1.        ,  1.38172324, 49.79531336],
       [ 1.        , 38.10050171, 34.68349946],
       [ 1.        ,  4.87107121,  2.0520299 ],
       [ 2.        ,  5.04739955, 13.42882308],
       [ 3.        , 33.57260343, 48.80832678],
       [ 2.        ,  5.00386314, 48.5601482 ],
       [ 1.        , 28.39419158, 31.14179231],
       [ 2.        , 48.65859258, 14.0780657 ],
       [ 1.        , 11.19492963, 43.78336613],
       [ 2.        , 18.04095606, 24.32833758],
       [ 1.        , 23.82980904, 28.1045774 ],
       [ 2.        ,  6.01496638, 34.923

In [ ]:
df = pd.DataFrame({
    "Node" : np.arange(1, 46),
    "weight": X[:][:,0],
    "x": X[:][:,1],
    "y": X[:][:,2]       
})

In [ ]:
df

,Node,weight,x,y
0,1,1.0,15.090687,43.360698
1,2,2.0,38.680856,15.351542
2,3,2.0,5.644361,32.985668
3,4,2.0,2.936060,9.053436
4,5,1.0,5.141422,49.565327
5,6,2.0,25.242105,49.389684
6,7,2.0,16.429767,47.934405
7,8,2.0,24.891321,48.135906
8,9,2.0,11.941441,43.608590
9,10,1.0,1.381723,49.795313


In [ ]:
df.iloc[0] = [1, 1.0, 25.0, 25.0]

In [ ]:
df

,Node,weight,x,y
0,1,1.0,25.000000,25.000000
1,2,2.0,38.680856,15.351542
2,3,2.0,5.644361,32.985668
3,4,2.0,2.936060,9.053436
4,5,1.0,5.141422,49.565327
5,6,2.0,25.242105,49.389684
6,7,2.0,16.429767,47.934405
7,8,2.0,24.891321,48.135906
8,9,2.0,11.941441,43.608590
9,10,1.0,1.381723,49.795313


In [ ]:
df.to_csv('export_dataframe.csv', index=False)